In [1]:
from random import choice
from enbios2.base.experiment import Experiment
import bw2data

In [2]:
bw2data.projects.set_current("ecoinvent")
exp = Experiment.create("ecoinvent")

2023-06-27 16:48:08,916 - enbios2.base.experiment - DEBUG - activity list


In [3]:
list(bw2data.databases)

['biosphere3',
 'apos_3.9.1_default',
 'consequential_3.9.1_default',
 'cutoff_3.9.1_default',
 'cutoff_3.9_default']

In [4]:
database_name = 'cutoff_3.9.1_default'
random_activity = bw2data.Database(database_name).random()
random_activity

'cyclohexane production' (kilogram, RoW, None)

In [5]:
exp.add_activity(random_activity, default_demand=[random_activity["unit"],1])

In [6]:
exp.activitiesMap

{'cyclohexane production': ExtendedExperimentActivityData(id=ExperimentActivityId(database='cutoff_3.9.1_default', code='7d5525311687b27493b6d616de7f7618', name=None, location=None, unit=None, alias='cyclohexane production'), output=ExtendedExperimentActivityOutput(unit='kilogram', magnitude=1.0), orig_id=None, bw_activity='cyclohexane production' (kilogram, RoW, None), scenario_outputs=None)}

In [7]:
exp.scenarios

[Scenario(experiment=<enbios2.base.experiment.Experiment object at 0x7f14527f9250>, alias='default scenario', activities_outputs={}, results=None, result_tree=None, methods=None)]

In [8]:
# this won't work because we don't have methods yet
exp.run()

ValueError: Scenario 'default scenario' has no methods

In [9]:
all_methods = list(bw2data.methods)
random_method = choice(all_methods)
random_method

('EF v3.1',
 'eutrophication: marine',
 'fraction of nutrients reaching marine end compartment (N)')

In [10]:
exp.add_method(random_method)

In [18]:
exp.run()

2023-06-27 16:48:41,824 - enbios2.base.experiment - INFO - Running scenario 'default scenario'


AttributeError: 'NoneType' object has no attribute 'items'

In [11]:
exp.methods

{'EF v3.1_eutrophication: marine_fraction of nutrients reaching marine end compartment (N)': ExperimentMethodData(id=('EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'), alias='EF v3.1_eutrophication: marine_fraction of nutrients reaching marine end compartment (N)')}